In [52]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import keras as k

In [53]:
df=pd.read_csv('./content/processedNew.csv')
df.shape

(67580, 82)

In [54]:
df['readmitted'].value_counts() 

0    39826
2    21625
1     6129
Name: readmitted, dtype: int64

In [57]:
feature_set = ['age', 'time_in_hospital', 'num_procedures', 'num_medications', 'number_diagnoses', 'metformin', 
                 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide',
                 'pioglitazone', 'rosiglitazone', 'acarbose', 'tolazamide', 'insulin', 'glyburide-metformin',
                 'AfricanAmerican', 'Asian', 'Caucasian', 'Hispanic', 'Other', 'gender_1', 
                 'admission_type_id_3', 'admission_type_id_5', 'discharge_disposition_id_2', 'discharge_disposition_id_7', 
                 'discharge_disposition_id_10', 'discharge_disposition_id_18', 'admission_source_id_4',
                 'admission_source_id_7', 'admission_source_id_9', 'max_glu_serum_0', 'max_glu_serum_1', 'A1Cresult_0',
                 'A1Cresult_1', 'num_medications|time_in_hospital', 'num_medications|num_procedures',
                 'time_in_hospital|num_lab_procedures', 'num_medications|num_lab_procedures', 'num_medications|number_diagnoses',
                 'age|number_diagnoses', 'change|num_medications', 'number_diagnoses|time_in_hospital',
                 'num_medications|numchange', 'level1_diag1_1.0', 'level1_diag1_2.0', 'level1_diag1_3.0', 'level1_diag1_4.0',
                 'level1_diag1_5.0','level1_diag1_6.0', 'level1_diag1_7.0', 'level1_diag1_8.0']

In [58]:
# from sklearn.feature_selection import mutual_info_classif
# import matplotlib.pyplot as plt
# importances=mutual_info_classif(X,df['readmitted'])
# feat_importances = pd.Series(importances)
# plt.figure().set_figheight(10)
# feat_importances.plot(kind='barh',color ="teal")
# plt.show()

In [59]:
# feature_set = ['age']

In [60]:
len(feature_set)

54

In [61]:
X = df[feature_set]
Y = pd.get_dummies(df['readmitted'])

In [62]:
X.shape

(67580, 54)

In [63]:
Y.shape

(67580, 3)

In [64]:
Y 

,0,1,2
0,0,0,1
1,1,0,0
2,1,0,0
3,1,0,0
4,0,0,1
...,...,...,...
67575,0,0,1
67576,0,0,1
67577,0,0,1
67578,1,0,0


In [65]:
from sklearn.model_selection import train_test_split
# X=df.drop('readmitted',axis=1)
# Y=df['readmitted']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

Random Forest Classfier

In [72]:
from sklearn.ensemble import RandomForestClassifier
# creating a RF classifier
clf = RandomForestClassifier(n_estimators = 100)  
  
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(X_train, y_train)
  
# performing predictions on the test dataset
y_pred = clf.predict(X_test)
  
# metrics are used to find accuracy or error
from sklearn import metrics 
  
# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))

ValueError: Found array with dim 3. RandomForestClassifier expected <= 2.

In [66]:
# Reshape input data for LSTM model
X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))
y_train = np.reshape(y_train.values, (y_train.shape[0], 1, y_train.shape[1]))
y_test = np.reshape(y_test.values, (y_test.shape[0], 1, y_test.shape[1]))

In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, ReLU, LeakyReLU
import tensorflow as tf
from tensorflow.keras.layers import Softmax
from keras.callbacks import EarlyStopping

In [68]:
model = Sequential()
model.add(Dense(1, input_shape=(1, X_train.shape[2])))
# model.add(LSTM(20, activation='relu'))
model.add(Dense(70,activation='relu'))
# model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.2))
model.add(Dense(70,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

In [69]:
optimizer = k.optimizers.Adam(learning_rate=0.001)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

# Fit the model to the training data
# , callbacks=[es]
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[es])

Epoch 1/20
1690/1690 [==============================] - 4s 2ms/step - loss: 0.9709 - accuracy: 0.5726 - val_loss: 0.8867 - val_accuracy: 0.5941
Epoch 2/20
1690/1690 [==============================] - 2s 1ms/step - loss: 0.8971 - accuracy: 0.5878 - val_loss: 0.8856 - val_accuracy: 0.5941
Epoch 3/20
1690/1690 [==============================] - 2s 1ms/step - loss: 0.8935 - accuracy: 0.5878 - val_loss: 0.8841 - val_accuracy: 0.5941
Epoch 4/20
1690/1690 [==============================] - 2s 1ms/step - loss: 0.8906 - accuracy: 0.5881 - val_loss: 0.8812 - val_accuracy: 0.5941
Epoch 5/20
1690/1690 [==============================] - 2s 1ms/step - loss: 0.8889 - accuracy: 0.5881 - val_loss: 0.8779 - val_accuracy: 0.5941
Epoch 6/20
1690/1690 [==============================] - 2s 1ms/step - loss: 0.8877 - accuracy: 0.5881 - val_loss: 0.8767 - val_accuracy: 0.5941
Epoch 7/20
1690/1690 [==============================] - 2s 1ms/step - loss: 0.8870 - accuracy: 0.5880 - val_loss: 0.8838 - val_accuracy:

In [70]:
# Evaluate the model on the test data
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8838340044021606
Test accuracy: 0.5941106677055359


In [71]:
score = model.evaluate(X_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

Train loss: 0.8959359526634216
Train accuracy: 0.588117778301239
